In [ ]:
!nvidia-smi

In [ ]:
%env PYTHONPATH=

In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX
conda install --channel defaults conda python=3.9 --yes
conda update --channel defaults --all --yes
conda --version
python --version

In [ ]:
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.9/site-packages"))

In [ ]:
!python -m pip install -r MIND-benchmarking-framework/requirements.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r 'drive/MyDrive/Colab Notebooks/Datasets/MIND/full' data

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/ranger/v3

In [ ]:
!python MIND-benchmarking-framework/src/main.py

In [ ]:
!cp /content/lightning_logs/ranger/v3/last.ckpt '/content/drive/MyDrive/Colab Notebooks/Checkpoints/MIND'